In [11]:
from datetime import datetime, timezone
import pandas as pd
from dyson_pool import DysonPool, Note_forward, Note_reverse


def format_number(number, precision=4):
    # Round to the specified precision
    rounded = round(number, precision)
    # Format with the specified precision and remove trailing zeros
    result = f"{rounded:.{precision}f}".rstrip("0").rstrip(".")
    return result


def print_pool_state(pool, label):
    snapshot = pool.snapshot(day=0, price=2000)  # Simplified day to a constant
    print(f"\n\U0001F4D8 {label} Pool State")
    df = pd.DataFrame([{
        "ETH Reserve": format_number(snapshot['reserve_eth']),
        "USDC Reserve": format_number(snapshot['reserve_usdc']),
        "K": format_number(snapshot['k']),
        "w": pool.w,
    }])
    print(df.to_markdown(index=False))

def print_note_summary(title, note):
    print(f"\n\U0001F4C4 {title}")
    if isinstance(note, Note_forward):
        data_items = [
            ("Input ETH", format_number(note.in0)),
            ("Input USDC", format_number(note.in1)),
            ("Note ETH", format_number(note.note0)),
            ("Note USDC", format_number(note.note1)),
            ("Price In", format_number(note.price_in)),
        ]
    elif isinstance(note, Note_reverse):
        data_items = [
            ("Target ETH (m)", format_number(note.m)),
            ("Target USDC (n)", format_number(note.n)),
            ("Delta X", format_number(note.delta_x)),
            ("Delta Y", format_number(note.delta_y)),
            ("Strike", format_number(note.strike)),
            ("Price In", format_number(note.price_in)),
            ("Revert Put (swap in, swap out)", f"({format_number(note.revert_put[0])}, {format_number(note.revert_put[1])})"),
            ("Revert Call (swap in, swap out)", f"({format_number(note.revert_call[0])}, {format_number(note.revert_call[1])})"),
        ]
    df = pd.DataFrame([dict(data_items)])
    print(df.to_markdown(index=False))

def print_withdraw_result(notes):
    print("\n\U0001F4B8 Withdraw Results")
    rows = [{
        "Note ID": n.id,
        "Returned ETH": format_number(amt0),
        "Returned USDC": format_number(amt1)
    } for n, amt0, amt1 in notes]
    print(pd.DataFrame(rows).to_markdown(index=False))

def test_forward_deposit():
    print("\n=== Testing Forward Dual Investment Deposit ===")
    pool = DysonPool(init_eth=100.0, init_usdc=200000.0, basis=0.5, w_factor=2)
    price = 2000

    print_pool_state(pool, "Initial")

    in0, in1 = 0, 2000
    nid = pool.deposit(in0, in1, price)
    note = pool.notes_forward[nid]  # Retrieve the Note_forward object

    print_note_summary("Forward Deposit Result", note)
    print_pool_state(pool, "After Forward Deposit")

def test_reverse_deposit_and_exercise():
    print("\n=== Testing Reverse Dual Investment Deposit and Revert ===")
    pool = DysonPool(init_eth=100.0, init_usdc=200000.0, basis=0.5, w_factor=1)
    price = 2000

    print_pool_state(pool, "Initial")

    m_n_values = [
        (1, 0, "put"),
        (0, 2000, "call"),
        (1, 2000, "put"),
        (1.1, 2000, "put")
    ]

    for m, n, option in m_n_values:
        pool = DysonPool(init_eth=100.0, init_usdc=200000.0, basis=0.5, w_factor=1)
        
        nid = pool.reverse_deposit(m, n, price)
        note = pool.notes_reverse[nid]  # Retrieve the Note_reverse object

        print_note_summary(f"Case m={m}, n={n}:", note)
        # print_pool_state(pool, f"After Case m={m}, n={n} Deposit")

        option_type, swap_in, swap_out = pool.revert_exercise(nid, option)
        if option_type == "put":
            print(f"\n\U0001F4B8 反悔行權 Put (swap1in: {swap_in}, swap0out: {swap_out})")
        elif option_type == "call":
            print(f"\n\U0001F4B8 反悔行權 Call (swap0in: {swap_in}, swap1out: {swap_out})")
        # print_pool_state(pool, case["after_exercise_label"])

# 正向雙幣

In [12]:
test_forward_deposit()


=== Testing Forward Dual Investment Deposit ===

📘 Initial Pool State
|   ETH Reserve |   USDC Reserve |       K |       w |
|--------------:|---------------:|--------:|--------:|
|           100 |         200000 | 4472.14 | 8944.27 |

📄 Forward Deposit Result
|   Input ETH |   Input USDC |   Note ETH |   Note USDC |   Price In |
|------------:|-------------:|-----------:|------------:|-----------:|
|           0 |         2000 |      0.995 |        2000 |       2000 |

📘 After Forward Deposit Pool State
|   ETH Reserve |   USDC Reserve |       K |       w |
|--------------:|---------------:|--------:|--------:|
|         100.5 |         201000 | 4494.44 | 8944.27 |


# 反向雙幣

In [13]:
test_reverse_deposit_and_exercise()


=== Testing Reverse Dual Investment Deposit and Revert ===

📘 Initial Pool State
|   ETH Reserve |   USDC Reserve |       K |       w |
|--------------:|---------------:|--------:|--------:|
|           100 |         200000 | 4472.14 | 4472.14 |

📄 Case m=1, n=0:
|   Target ETH (m) |   Target USDC (n) |   Delta X |   Delta Y |   Strike |   Price In | Revert Put (swap in, swap out)   | Revert Call (swap in, swap out)   |
|-----------------:|------------------:|----------:|----------:|---------:|-----------:|:---------------------------------|:----------------------------------|
|                1 |                 0 |         1 |   -1980.2 |   1980.2 |       2000 | (1980.198, 1)                    | (0, 0)                            |

💸 反悔行權 Put (swap1in: 1980.1980198019803, swap0out: 1)

📄 Case m=0, n=2000:
|   Target ETH (m) |   Target USDC (n) |   Delta X |   Delta Y |   Strike |   Price In | Revert Put (swap in, swap out)   | Revert Call (swap in, swap out)   |
|-----------------: